In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
import yaml
import pandas as pd
from pathlib import Path

from utils.logging_utils import ExperimentLogger
from training_routine.trainer import Trainer
from pipeline.walkforward import WFCVGenerator
from pipeline.wf_config import WFConfig
from models.mlp import build_model
from utils.paths import CONFIG_DIR
from utils.gpu_test import gpu_test
from utils.custom_formatter import setup_logger



# -------- argparse setup --------


# setup logger
info_logger = setup_logger("Experiment", level="INFO")

# --- run GPU test once ---
gpu_test()
info_logger.info("GPU check complete.")


# -------- load config --------
cfg = yaml.safe_load(open(CONFIG_DIR / "debug.yaml"))

# -------- data + components --------
info_logger = ExperimentLogger(cfg["experiment"]["name"], cfg)

# use data path if present
data_path = cfg.get("data", {}).get("df_path")

# import wfcv
wf_config = WFConfig(**cfg["walkforward"])

if data_path:
    df = pd.read_parquet(data_path)
    wf = WFCVGenerator(df_long=df, config=wf_config)
else:
    wf = WFCVGenerator(config=wf_config)

# instantiate trainer    
trainer = Trainer(cfg, info_logger)

# Only number of rows change, columns stay constant 
input_shape = cfg["walkforward"]["lags"]

max_folds = cfg["walkforward"]["max_folds"]

# -------- train per fold --------
for fold, data in enumerate(wf.folds()):
    if fold > 0:
        break  # allow running subset of folds

    # can i take this out the for loop or do i risk leakage?
    # better to leave it here so im sure weights are initialized 
    # at each fold
    #model = build_model(cfg["model"]["hparams"], input_shape)

    #trainer.fit_eval_fold(model, data, trial=0, fold=fold)

data 

2025-10-12 16:42:28 - GPU-Test - DEBUG - ==================== Running GPU test ==================== (gpu_test.py:15)
2025-10-12 16:42:28 - GPU-Test - DEBUG - TensorFlow version: 2.18.1 (gpu_test.py:16)
2025-10-12 16:42:28 - GPU-Test - DEBUG - Visible GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')] (gpu_test.py:19)
2025-10-12 16:42:28 - GPU-Test - DEBUG - Built with CUDA?: True (gpu_test.py:22)
2025-10-12 16:42:28 - GPU-Test - DEBUG - Build info:
OrderedDict({'cpu_compiler': '/usr/lib/llvm-18/bin/clang', 'cuda_compute_capabilities': ['sm_60', 'sm_70', 'sm_80', 'sm_89', 'compute_90'], 'cuda_version': '12.5.1', 'cudnn_version': '9', 'is_cuda_build': True, 'is_rocm_build': False, 'is_tensorrt_build': False}) (gpu_test.py:25)
2025-10-12 16:42:28 - GPU-Test - DEBUG - libcuda.so.1 is loadable ✅ (gpu_test.py:29)
2025-10-12 16:42:28 - GPU-Test - DEBUG - GPU test completed successfully. (gpu_test.py:34)
2025-10-12 16:42:28 - Experiment - INFO - GPU check complete. (44782

(array([[10078., 10078., 10078., ..., 10078., 10078., 10078.],
        [10104., 10104., 10104., ..., 10104., 10104., 10104.],
        [10107., 10107., 10107., ..., 10107., 10107., 10107.],
        ...,
        [90609., 90609., 90609., ..., 90609., 90609., 90609.],
        [91380., 91380., 91380., ..., 91380., 91380., 91380.],
        [92655., 92655., 92655., ..., 92655., 92655., 92655.]],
       shape=(365258, 20)),
 array([10078., 10104., 10107., ..., 90609., 91380., 92655.],
       shape=(365258,)),
 array([[10078., 10078., 10078., ..., 10078., 10078., 10078.],
        [10104., 10104., 10104., ..., 10104., 10104., 10104.],
        [10107., 10107., 10107., ..., 10107., 10107., 10107.],
        ...,
        [90609., 90609., 90609., ..., 90609., 90609., 90609.],
        [91380., 91380., 91380., ..., 91380., 91380., 91380.],
        [92655., 92655., 92655., ..., 92655., 92655., 92655.]],
       shape=(115175, 20)),
 array([10078., 10104., 10107., ..., 90609., 91380., 92655.],
       shap

In [6]:
data

(array([[10078., 10078., 10078., ..., 10078., 10078., 10078.],
        [10104., 10104., 10104., ..., 10104., 10104., 10104.],
        [10107., 10107., 10107., ..., 10107., 10107., 10107.],
        ...,
        [90609., 90609., 90609., ..., 90609., 90609., 90609.],
        [91380., 91380., 91380., ..., 91380., 91380., 91380.],
        [92655., 92655., 92655., ..., 92655., 92655., 92655.]],
       shape=(365258, 20)),
 array([10078., 10104., 10107., ..., 90609., 91380., 92655.],
       shape=(365258,)),
 array([[10078., 10078., 10078., ..., 10078., 10078., 10078.],
        [10104., 10104., 10104., ..., 10104., 10104., 10104.],
        [10107., 10107., 10107., ..., 10107., 10107., 10107.],
        ...,
        [90609., 90609., 90609., ..., 90609., 90609., 90609.],
        [91380., 91380., 91380., ..., 91380., 91380., 91380.],
        [92655., 92655., 92655., ..., 92655., 92655., 92655.]],
       shape=(115175, 20)),
 array([10078., 10104., 10107., ..., 90609., 91380., 92655.],
       shap

In [ ]:

W = (wf.df.pivot(index=id_col, columns=t_col, values=wf.id_col)
                    .sort_index(axis=1))

,t,ret,permno
0,0,-0.012107,10078
1,1,-0.062092,10078
2,2,0.001742,10078
3,3,-0.053913,10078
4,4,0.056985,10078
...,...,...,...
155298,6284,0.073572,93436
155299,6285,-0.01763,93436
155300,6286,-0.049479,93436
155301,6287,-0.033012,93436
